In [29]:
import numpy as np
from numpy.core.fromnumeric import shape
import pandas as pd
import datetime as dt
import glob
import pickle
import global_variables as gv
import functions as f
import random
import matplotlib.pyplot as plt

In [41]:
# Import journey and price data

journeys_range = pickle.load(open('Outputs/journeys_range','rb'))
price_data = pickle.load(open('Outputs/price_data','rb'))


In [43]:
journeys_range

Branch_ID Start_Time_of_Route   End_Time_of_Route  \
date       Route_ID                                                      
2019-02-10 2104108         513 2019-02-10 05:27:00 2019-02-10 12:35:00   
           2104116         513 2019-02-10 05:42:00 2019-02-10 13:14:00   
           2104120         513 2019-02-10 13:29:00 2019-02-10 20:17:59   
           2104123         513 2019-02-10 13:15:00 2019-02-10 21:24:59   
           2104125         513 2019-02-10 13:15:00 2019-02-10 22:02:00   
...                        ...                 ...                 ...   
2019-02-17 2112902         513 2019-02-17 06:46:59 2019-02-17 13:40:00   
           2112903         513 2019-02-17 06:14:59 2019-02-17 13:30:00   
           2112906         513 2019-02-17 06:23:00 2019-02-17 13:51:00   
           2112913         513 2019-02-17 06:42:00 2019-02-17 14:11:00   
           2112914         513 2019-02-17 06:47:59 2019-02-17 14:01:00   

                     Energy_Required  Vehicle_ID      Next_Departure  \
date       Route_ID                                                    
2019-02-10 2104108         26.571267           3 2019-02-10 13:15:00   
           2104116         23.569667           4 2019-02-10 13:15:00   
           2104120         26.296093          12 2019-02-11 05:45:00   
           2104123         27.635793           3 2019-02-11 05:15:00   
           2104125         30.475507           4 2019-02-11 05:15:00   
...                              ...         ...                 ...   
2019-02-17 2112902         19.250487          22 2019-02-17 15:45:00   
           2112903         25.309860          12 2019-02-17 14:24:00   
           2112906         25.225453          16 2019-02-17 14:45:00   
           2112913         26.972127          20 2019-02-17 15:23:59   
           2112914         37.156593          23 2019-02-17 16:02:00   

                       Previous_Arrival  
date       Route_ID                      
2019-02-10 2104108  2019-02-09 21:51:00  
           2104116  2019-02-09 21:35:00  
           2104120  2019-02-10 12:46:00  
           2104123  2019-02-10 12:35:00  
           2104125  2019-02-10 13:14:00  
...                                 ...  
2019-02-17 2112902  2019-02-16 14:05:00  
           2112903  2019-02-16 13:24:00  
           2112906  2019-02-16 13:46:00  
           2112913  2019-02-16 13:33:59  
           2112914  2019-02-16 14:26:00  

[142 rows x 7 columns]

In [45]:
# Create function for one route, in one day
def singleroute_BAU_schedule(journeys, day, route, eprice):
    mask = ( (eprice['from'] >= dt.datetime.combine(day, gv.CHAR_ST)) 
    & (eprice['from'] < dt.datetime.combine(day + dt.timedelta(days=1), gv.CHAR_ST)))
    single_profile = eprice[mask][['from','Electricity_Price','Time_Price']].copy()
    single_profile['Route_ID'] = route
    single_profile['Vehicle_ID'] = journeys.loc[(day,route)]['Vehicle_ID']
    return single_profile

def BAU_charging(journeys, eprice): # TODO change to empty charging
    # Create df for charge profile, with time slots in that time range. 
    _, time_range = f.create_empty_schedule(journeys, eprice) # TODO Make this efficient
    # Iterate over each day
    dates = journeys.index.unique(level='date')
    # print(dates)
    day_profile = {}
    vehicles = journeys['Vehicle_ID'].unique()
    for date in dates:
        day = date.to_pydatetime()
        if day.date() == time_range[1].date():
            break
        # Iterate over routes, copy to correct column
        route_profiles = {}
        for route in journeys.loc[day].index:
            route_profiles[route] = singleroute_BAU_schedule(
                journeys, 
                day, 
                route, 
                eprice)
        day_profile[day] = pd.concat(list(route_profiles.values()))
    profiles = pd.concat(list(day_profile.values()))
    profiles.sort_values(by=['from','Route_ID'],inplace=True)
    profiles.set_index(['from','Route_ID'],inplace=True)
    return profiles

In [46]:
BAU_profile = BAU_charging(journeys_range, price_data)
pickle.dump(BAU_profile,open('Outputs/BAU_profile','wb'))

In [40]:
BAU_profile

Electricity_Price  Time_Price  Vehicle_ID
from                Route_ID                                           
2019-02-10 11:00:00 2104108               12.26         502           3
                    2104116               12.26         502           4
                    2104120               12.26         502          12
                    2104123               12.26         502           3
                    2104125               12.26         502           4
...                                         ...         ...         ...
2019-02-17 10:30:00 2111880               10.40         837          23
                    2111882               10.40         837          20
                    2111887               10.40         837          16
                    2111889               10.40         837          14
                    2111897               10.40         837          22

[5856 rows x 3 columns]